In [9]:
from supabase import create_client
import pandas as pd
import utils.env as env

url = env.SUPABASE_URL
key = env.SUPABASE_KEY
supabase = create_client(url, key)


def _get_all_album_statistics(artist_uri):
    album_response = supabase.table("albums")\
        .select("album_name, metrics(popularity, danceability, energy, valence)")\
        .like('artist_uris', f'%{artist_uri}%')\
        .execute()
    # convert all data to DF; then return
    return album_response.data

response = _get_all_album_statistics("spotify:artist:13ubrt8QOOCPljQ2FL1Kca")
for idx, album in enumerate(response):
    name = response[idx].get('album_name')
    #metrics = response.get('metrics')
print(name)


2024-01-22 10:32:45,872:INFO - HTTP Request: GET https://snuqfngjrznygycntzst.supabase.co/rest/v1/albums?select=album_name%2C%20metrics%28popularity%2C%20danceability%2C%20energy%2C%20valence%29&artist_uris=like.%25spotify%3Aartist%3A13ubrt8QOOCPljQ2FL1Kca%25 "HTTP/1.1 200 OK"


TESTING [{'energy': 0.869, 'valence': 0.082, 'popularity': 55, 'danceability': 0.655}, {'energy': 0.754, 'valence': 0.394, 'popularity': 63, 'danceability': 0.428}, {'energy': 0.783, 'valence': 0.33, 'popularity': 57, 'danceability': 0.512}, {'energy': 0.412, 'valence': 0.304, 'popularity': 67, 'danceability': 0.798}, {'energy': 0.569, 'valence': 0.294, 'popularity': 83, 'danceability': 0.854}, {'energy': 0.403, 'valence': 0.463, 'popularity': 52, 'danceability': 0.514}, {'energy': 0.85, 'valence': 0.58, 'popularity': 55, 'danceability': 0.433}, {'energy': 0.632, 'valence': 0.715, 'popularity': 56, 'danceability': 0.852}, {'energy': 0.398, 'valence': 0.437, 'popularity': 49, 'danceability': 0.57}, {'energy': 0.477, 'valence': 0.313, 'popularity': 54, 'danceability': 0.91}, {'energy': 0.669, 'valence': 0.127, 'popularity': 59, 'danceability': 0.302}, {'energy': 0.546, 'valence': 0.355, 'popularity': 59, 'danceability': 0.584}, {'energy': 0.326, 'valence': 0.338, 'popularity': 57, 'dance